## Рекомендательные системы
### Курсовой проект

In [1]:
import pandas as pd
import numpy as np
import pickle

from scipy.sparse import csr_matrix
from implicit import als

import lightgbm as lgb
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

### 1. Загрузка данных

In [3]:
PATH = 'C:/Users/ASER/Desktop/GeekBrains/RecSys/data/'

In [4]:
data = pd.read_csv(PATH + 'retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
test = pd.read_csv(PATH + 'retail_test1.csv')
test.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0


In [6]:
item_features = pd.read_csv(PATH + 'product.csv')
user_features = pd.read_csv(PATH + 'hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

In [7]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [8]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


### 2. Разбивка данных на обучающие, валидационные и тестовые

#### Схема обучения и валидации:
* -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 

In [9]:
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

#### Появление новых пользователей на валидационной и тестовой выборках

In [10]:
users_lvl_1 = data_train_lvl_1.user_id.unique()
users_lvl_2 = data_val_lvl_1.user_id.unique()
users_lvl_3 = data_val_lvl_2.user_id.unique()

new_users_lvl_2 = list(set(users_lvl_2) - set(users_lvl_1))
new_users_lvl_3 = list(set(users_lvl_3) - (set(users_lvl_1) | set(users_lvl_2)))

add_to_lvl_2 = list(set(users_lvl_3) - (set(users_lvl_2)))

new_users_lvl_2, new_users_lvl_3, len(add_to_lvl_2)

([1984], [], 126)

### 3. Уменьшение номенклатуры товаров

In [11]:
n_items_before = data_train_lvl_1['item_id'].nunique()
data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)
n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


### 4. Генерирование новых признаков для модели второго уровня

#### 4.1 Признаки пользователей

In [12]:
user_features['age_desc'].replace(
    {'19-24': 22, '25-34': 30, '35-44': 40, '45-54': 50, '55-64': 60, '65+': 70},
    inplace=True)

user_features['marital_status_code'].replace(
    {'U': 0, 'A': 1, 'B': 2}, inplace=True)

user_features['income_desc'].replace(
    {'Under 15K': 10, '15-24K': 20, '25-34K':30, '35-49K': 40,
     '50-74K': 62, '75-99K': 87, '100-124K': 112, '125-149K': 137, 
     '150-174K': 162, '175-199K': 187, '200-249K': 225, '250K+':275}, inplace=True)

user_features['homeowner_desc'].replace(
    {'Unknown': 0, 'Probable Renter': 1, 'Renter': 2,
     'Probable Owner': 3, 'Homeowner': 4}, inplace=True)

user_features['hh_comp_desc'].replace(
    {'Unknown': 0, 'Single Male': 1, 'Single Female': 2,
     '1 Adult Kids': 3, '2 Adults No Kids': 4, '2 Adults Kids':5},inplace=True)

user_features['household_size_desc'].replace({'5+': 5}, inplace=True) 

user_features['kid_category_desc'].replace(
    {'None/Unknown': 0, '3+': 3}, inplace=True)

user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,70,1,40,4,4,2,0,1
1,50,1,62,4,4,2,0,7


#### 4.2 Признаки товаров

In [13]:
names = ['manufacturer', 'department', 'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product']
for name in names:
    new_name = name + '_freq'
    a = item_features[name].value_counts()
    ind = a.index.tolist()
    for i in ind:
        item_features.loc[item_features[name] == i, new_name] = a[i]

item_features['brand'] = np.where(item_features['brand']=='Private', 0, 1)

commodities = item_features.commodity_desc.value_counts()
commodities_list = commodities.keys().tolist()
for i, name in enumerate(commodities_list):
    item_features.loc[item_features['commodity_desc'] == name, 'commodity_category'] = i

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,manufacturer_freq,department_freq,commodity_desc_freq,sub_commodity_desc_freq,curr_size_of_product_freq,commodity_category
0,25671,2,GROCERY,1,FRZN ICE,ICE - CRUSHED/CUBED,22 LB,1411.0,39021.0,29.0,29.0,12.0,269.0
1,26081,2,MISC. TRANS.,1,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,,1411.0,490.0,490.0,429.0,30607.0,56.0


#### 4.3 Новые признаки пользователей и товаров на основе матрицы транзакций

In [14]:
def get_user_item_features(data_train_lvl_1):
    # час совершения транзакции
    X = data_train_lvl_1.copy()
    X['hour'] = X['trans_time'] // 100
    user_item_features = X.groupby(['user_id', 'item_id'])['hour'].median().reset_index()
    user_item_features.columns = ['user_id', 'item_id', 'median_sales_hour']
    
    # день недели совершения транзакции
    X['weekday'] = X['day'] % 7
    df = X.groupby(['user_id', 'item_id'])['weekday'].median().reset_index()
    df.columns = ['user_id', 'item_id', 'median_weekday']
    user_item_features = user_item_features.merge(df, on=['user_id', 'item_id'])
    
    # cреднее кол-во дней между покупками
    df = X.groupby('user_id')['day'].nunique().reset_index()
    df['mean_visits_interval'] = (X.groupby('user_id')['day'].max() - X.groupby('user_id')['day'].min()) / df['day']
    user_item_features = user_item_features.merge(df[['user_id', 'mean_visits_interval']], on=['user_id'])
    
    # средний чек корзины клиента
    df = X.groupby(['user_id', 'basket_id'])['sales_value'].sum().reset_index()
    df = df.groupby('user_id')['sales_value'].mean().reset_index()
    df.columns = ['user_id', 'mean_check']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # кол-во магазинов, в которых продавался товар
    df = X.groupby(['item_id'])['store_id'].nunique().reset_index()
    df.columns = ['item_id', 'n_stores']
    user_item_features = user_item_features.merge(df, on=['item_id'])
    
    # кол-во уникальных товаров, купленных клиентом
    df = X.groupby(['user_id'])['item_id'].nunique().reset_index()
    df.columns = ['user_id', 'n_items']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # кол-во транзакций клиента
    df = X.groupby(['user_id'])['item_id'].count().reset_index()
    df.columns = ['user_id', 'n_transactions']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # mean / max / std кол-ва уникальных товаров в корзине клиента
    df = X.groupby(['user_id', 'basket_id'])['item_id'].nunique().reset_index()
    df1 = df.groupby('user_id')['item_id'].mean().reset_index()
    df1.columns = ['user_id', 'mean_n_items_basket']
    user_item_features = user_item_features.merge(df1, on=['user_id'])

    df2 = df.groupby('user_id')['item_id'].max().reset_index()
    df2.columns = ['user_id', 'max_n_items_basket']
    user_item_features = user_item_features.merge(df2, on=['user_id'])

    df3 = df.groupby('user_id')['item_id'].std().reset_index()
    df3.columns = ['user_id', 'std_n_items_basket']
    user_item_features = user_item_features.merge(df3, on=['user_id'])
    
    # mean / max / std кол-ва уникальных категорий в корзине клиента
    X = X.merge(item_features[['item_id', 'commodity_desc']], on=['item_id'])
    df = X.groupby(['user_id', 'basket_id'])['commodity_desc'].nunique().reset_index()
    df1 = df.groupby('user_id')['commodity_desc'].mean().reset_index()
    df1.columns = ['user_id', 'mean_n_item_categories_basket']
    user_item_features = user_item_features.merge(df1, on=['user_id'])

    df2 = df.groupby('user_id')['commodity_desc'].max().reset_index()
    df2.columns = ['user_id', 'max_n_item_categories_basket']
    user_item_features = user_item_features.merge(df2, on=['user_id'])

    df3 = df.groupby('user_id')['commodity_desc'].std().reset_index()
    df3.columns = ['user_id', 'std_n_item_categories_basket']
    user_item_features = user_item_features.merge(df3, on=['user_id'])
    
    # эмбеддинги товаров
    recommender = MainRecommender(X)
    df = recommender.model.item_factors
    n_factors = recommender.model.factors
    ind = list(recommender.id_to_itemid.values())
    df = pd.DataFrame(df, index=ind).reset_index()
    df.columns = ['item_id'] + ['factor_' + str(i + 1) for i in range(n_factors)]
    user_item_features = user_item_features.merge(df, on=['item_id'])
    
    # эмбеддинги пользователей
    df = recommender.model.user_factors
    ind = list(recommender.id_to_userid.values())
    df = pd.DataFrame(df, index=ind).reset_index()
    df.columns = ['user_id'] + ['user_factor_' + str(i + 1) for i in range(n_factors)]
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    return user_item_features

In [15]:
user_item_features = get_user_item_features(data_train_lvl_1)
user_item_features.head(2)

,user_id,item_id,median_sales_hour,median_weekday,mean_visits_interval,mean_check,n_stores,n_items,n_transactions,mean_n_items_basket,...,user_factor_11,user_factor_12,user_factor_13,user_factor_14,user_factor_15,user_factor_16,user_factor_17,user_factor_18,user_factor_19,user_factor_20
0,1,820165,13.0,2.0,NaN,54.534677,103,218,1355,12.693548,...,-1.71888,-4.212374,-11.551077,-5.481315,8.41078,8.169738,12.084761,18.812346,21.233715,3.335375
1,1,823721,13.0,4.0,NaN,54.534677,95,218,1355,12.693548,...,-1.71888,-4.212374,-11.551077,-5.481315,8.41078,8.169738,12.084761,18.812346,21.233715,3.335375


### 5. Формирование базы данных для обучения модели второго уровня

In [16]:
def get_candidates(data_train_lvl_1, data_train_lvl_2, N, add_to_lvl_2):
    recommender = MainRecommender(data_train_lvl_1)

    users_lvl_1 = data_train_lvl_1['user_id'].unique()
    users_lvl_2 = data_train_lvl_2['user_id'].unique().tolist()
    if add_to_lvl_2:
        users_lvl_2 += add_to_lvl_2

    current_users = list(set(users_lvl_2) & set(users_lvl_1))    
    new_users = list(set(users_lvl_2) - set(users_lvl_1))

    df = pd.DataFrame(users_lvl_2, columns=['user_id'])
    cond_1 = df['user_id'].isin(current_users)
    df.loc[cond_1, 'candidates'] = df.loc[cond_1, 'user_id'].apply(
        lambda x: recommender.get_own_recommendations(x, N))

    if new_users:
        cond_2 = df['user_id'].isin(new_users)
        df.loc[cond_2, 'candidates'] = df.loc[cond_2, 'user_id'].apply(
            lambda x: recommender.overall_top_purchases[:N])
        
    return df

In [17]:
def get_targets_lvl_2(data_train_lvl_1, data_train_lvl_2, user_item_features, N, add_to_lvl_2=None):
    
    users_lvl_2 = get_candidates(data_train_lvl_1, data_train_lvl_2, N, add_to_lvl_2)
    
    df = pd.DataFrame({'user_id': users_lvl_2['user_id'].values.repeat(N),
                       'item_id': np.concatenate(users_lvl_2['candidates'].values)})

    targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
    targets_lvl_2['target'] = 1  # тут только покупки 

    targets_lvl_2 = df.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')
    targets_lvl_2['target'].fillna(0, inplace= True)
    
    targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
    targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')
    targets_lvl_2 = targets_lvl_2.merge(
        user_item_features, on=['user_id', 'item_id'], how='left')
    
    return targets_lvl_2

In [18]:
N = 500
targets_lvl_2 = get_targets_lvl_2(data_train_lvl_1, data_train_lvl_2, user_item_features, N, add_to_lvl_2)

print(f'число пользователей: {targets_lvl_2.user_id.nunique()}')
print(f'среднее число покупок: {round(targets_lvl_2["target"].mean(), 4)}')

targets_lvl_2.head(2)


число пользователей: 2280
среднее число покупок: 0.065


,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,manufacturer_freq,...,user_factor_11,user_factor_12,user_factor_13,user_factor_14,user_factor_15,user_factor_16,user_factor_17,user_factor_18,user_factor_19,user_factor_20
0,2070,834103,1.0,2224,GROCERY,1,SOFT DRINKS,SFT DRNK SNGL SRV BTL CARB (EX,20 OZ,355.0,...,2.097624,1.355374,4.569934,0.218316,-3.118889,5.294017,-2.700953,0.114237,7.39686,12.564788
1,2070,834103,1.0,2224,GROCERY,1,SOFT DRINKS,SFT DRNK SNGL SRV BTL CARB (EX,20 OZ,355.0,...,2.097624,1.355374,4.569934,0.218316,-3.118889,5.294017,-2.700953,0.114237,7.39686,12.564788


### 6. Отбор признаков

In [19]:
SELECTED_FEATURES_NAMES = ['brand',
                           'manufacturer_freq', 'department_freq', 'commodity_desc_freq',
                           'sub_commodity_desc_freq', 'curr_size_of_product_freq',
                           'commodity_category',
                           'age_desc', 'marital_status_code', 'income_desc', 
                           'homeowner_desc', 'hh_comp_desc',
                            
                           'marital_status_code', 'homeowner_desc', 
                           'hh_comp_desc', 'manufacturer',                            
                           
                           'median_sales_hour', 'median_weekday', #'mean_visits_interval',
                           'mean_check', 
                           'n_stores', 'n_items', 'n_transactions', 
                           'mean_n_items_basket', 'max_n_items_basket', 'std_n_items_basket',
                           'mean_n_item_categories_basket', 'max_n_item_categories_basket', 
                           'std_n_item_categories_basket',
                           'factor_1', 'factor_2', 'factor_3', 'factor_4', 'factor_5',
                           'factor_6', 'factor_7', 'factor_8', 'factor_9', 'factor_10',
                           'factor_11', 'factor_12', 'factor_13', 'factor_14', 'factor_15',
                           'factor_16', 'factor_17', 'factor_18', 'factor_19', 'factor_20',
                           
                           'user_factor_1', 'user_factor_2', 'user_factor_3', 'user_factor_4',
                           'user_factor_5', 'user_factor_6', 'user_factor_7', 'user_factor_8',
                           'user_factor_9', 'user_factor_10', 'user_factor_11', 'user_factor_12',
                           'user_factor_13', 'user_factor_14', 'user_factor_15', 'user_factor_16',
                           'user_factor_17', 'user_factor_18', 'user_factor_19', 'user_factor_20',                           
                          ]
categorical = ['marital_status_code','homeowner_desc', 
               'hh_comp_desc', 'manufacturer',
               'commodity_category'
              ]

### 7. Модели

#### CatBoost

In [20]:
SELECTED_FEATURES_NAMES_cb = [i for i in SELECTED_FEATURES_NAMES if not i in categorical]

In [21]:
def run_model_cb(targets_lvl_2):    
    X_train, X_valid, y_train, y_valid = train_test_split(targets_lvl_2[SELECTED_FEATURES_NAMES_cb].fillna(0),
                                                          targets_lvl_2[['target']],
                                                          test_size=0.2, random_state=16,
                                                          stratify=targets_lvl_2[['target']])

    dtrain = Pool(data=X_train, label=y_train) #, cat_features=categorical
    dvalid = Pool(data=X_valid, label=y_valid) #, cat_features=categorical

    params_cb = {"n_estimators":5000,
                 "learning_rate": 0.1,
                 "loss_function": "Logloss",
                 "eval_metric": "AUC",
                 "task_type": "CPU",
                 "max_bin": 30,
                 "early_stopping_rounds": 30,
                 "verbose": 1000,
                 "max_depth": 10,
                 "l2_leaf_reg": 80,
                 "thread_count": 6,
                 "random_seed": 51} 

    model_cb = CatBoostClassifier(**params_cb)
    model_cb.fit(dtrain, eval_set=[dvalid])

    
    return model_cb

#### LightGBM

In [22]:
def run_model_lgb(targets_lvl_2):
    X_train, X_valid, y_train, y_valid = train_test_split(targets_lvl_2[SELECTED_FEATURES_NAMES].fillna(0),
                                                          targets_lvl_2[['target']],
                                                          test_size=0.2, random_state=16,
                                                          stratify=targets_lvl_2[['target']])

    dtrain = lgb.Dataset(X_train, y_train, categorical_feature=categorical)
    dvalid = lgb.Dataset(X_valid, y_valid, categorical_feature=categorical)

    params_lgb = {"boosting_type": "gbdt",
                  "objective": "binary", 
                  "metric": "auc",
                  "num_boost_round": 10000,
                  "learning_rate": 0.1,
                  "class_weight": 'balanced',
                  "max_depth": 10,
                  "n_estimators": 5000,
                  "n_jobs": 6,
                  "seed": 12} 

    model_lgb = lgb.train(params=params_lgb,
                          train_set=dtrain,  
                          valid_sets=[dtrain, dvalid],
                          categorical_feature=categorical,
                          verbose_eval=1000,
                          early_stopping_rounds=30)
    
    return model_lgb

#### Генерация прогнозов

In [23]:
model_lgb = run_model_lgb(targets_lvl_2)
# [2426]	training's auc: 0.964429	valid_1's auc: 0.896633

Training until validation scores don't improve for 30 rounds
[1000]	training's auc: 0.940784	valid_1's auc: 0.889137
[2000]	training's auc: 0.960094	valid_1's auc: 0.895502
Early stopping, best iteration is:
[2426]	training's auc: 0.964429	valid_1's auc: 0.896633


In [24]:
model_cb = run_model_cb(targets_lvl_2)
# bestTest = 0.9069283804
# bestIteration = 2598

0:	test: 0.7776645	best: 0.7776645 (0)	total: 1.81s	remaining: 2h 31m 2s
1000:	test: 0.8952545	best: 0.8952545 (1000)	total: 26m 26s	remaining: 1h 45m 36s
2000:	test: 0.9051401	best: 0.9051401 (2000)	total: 49m 27s	remaining: 1h 14m 8s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.9069283804
bestIteration = 2598

Shrink model to first 2599 iterations.


In [25]:
predictions_lgb_train = model_lgb.predict(targets_lvl_2[SELECTED_FEATURES_NAMES].fillna(0))
predictions_cb_train = model_cb.predict_proba(targets_lvl_2[SELECTED_FEATURES_NAMES_cb].fillna(0))[:, 1]

Объединение прогнозов

In [26]:
preds_train = pd.DataFrame(zip(predictions_lgb_train, 
                               predictions_cb_train),
                           columns=['lgb', 'cb']).mean(axis=1).values
roc_auc_score(targets_lvl_2['target'], preds_train)
# 0.9557067627903868

0.9557067627903868

### 8. Обработка прогнозов

In [27]:
def get_predictions(targets_lvl_2, raw_predictions, prefix='lgb'): 
    df = targets_lvl_2[['user_id', 'item_id']]
    df['predictions'] = raw_predictions

    df = df.groupby(['user_id', 'item_id'])['predictions'].median().reset_index()
    df = df.sort_values(['predictions'], ascending=False).groupby(['user_id']).head(5)

    df = df.groupby('user_id')['item_id'].unique().reset_index()
    df.columns = ['user_id', prefix + '_recommendations']
    
    return df

In [28]:
def get_results(data_val_lvl_2, targets_lvl_2, preds_lgb, preds_cb, combined_preds):
    result = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
    result.columns=['user_id', 'actual']
    
    prefixes = ['lgb', 'cb', 'cb_lgb']
    predictions = [preds_lgb, preds_cb, combined_preds]
    
    for i, preds in enumerate(predictions):
        df = get_predictions(targets_lvl_2, preds, prefixes[i])
        result = result.merge(df, on='user_id', how='left')

    return result

In [29]:
result_lvl_2 = get_results(data_val_lvl_2, targets_lvl_2, 
                           predictions_lgb_train,
                           predictions_cb_train, 
                           preds_train)
result_lvl_2.head(2)

,user_id,actual,lgb_recommendations,cb_recommendations,cb_lgb_recommendations
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1082185, 940947, 8293439, 9297615, 995242]","[1082185, 8293439, 872137, 940947, 995242]","[1082185, 8293439, 940947, 995242, 9297615]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 1101010, 1053690, 1092026, 951590]","[5569230, 1082185, 983584, 1092026, 13842214]","[1106523, 5569230, 1092026, 1101010, 983584]"


### 9. Метрики качества

#### LightGBM

In [30]:
result_lvl_2.apply(lambda row: precision_at_k(row['lgb_recommendations'], row['actual'], 5), axis=1).mean()
# 0.33702252693437806

0.33702252693437806

#### CatBoost

In [31]:
result_lvl_2.apply(lambda row: precision_at_k(row['cb_recommendations'], row['actual'], 5), axis=1).mean()
# 0.3584720861900098

0.3584720861900098

#### Ensemble

In [32]:
result_lvl_2.apply(lambda row: precision_at_k(row['cb_lgb_recommendations'], row['actual'], 5), axis=1).mean()
# 0.3587659157688541

0.3587659157688541

### 10. Построение финальных прогнозов

Финальные прогнозы строятся на всей обучающей выборке

In [33]:
validation_weeks = 6
data_train = data[data['week_no'] < data['week_no'].max() - validation_weeks]
data_valid = data[data['week_no'] >= data['week_no'].max() - validation_weeks]

In [34]:
users_lvl_1 = data_train.user_id.unique()
users_lvl_2 = data_valid.user_id.unique()
users_lvl_3 = test.user_id.unique()

new_users_lvl_2 = list(set(users_lvl_2) - set(users_lvl_1))
new_users_lvl_3 = list(set(users_lvl_3) - (set(users_lvl_1) | set(users_lvl_2)))

add_to_lvl_2 = list(set(users_lvl_3) - (set(users_lvl_2)))

new_users_lvl_2, new_users_lvl_3, len(add_to_lvl_2)

([], [2325], 75)

In [35]:
n_items_before = data['item_id'].nunique()
data_train = prefilter_items(data_train, item_features=item_features, take_n_popular=5000)
n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 89051 to 5001


In [36]:
user_item_features = get_user_item_features(data_train)
user_item_features.head(2)

,user_id,item_id,median_sales_hour,median_weekday,mean_visits_interval,mean_check,n_stores,n_items,n_transactions,mean_n_items_basket,...,user_factor_11,user_factor_12,user_factor_13,user_factor_14,user_factor_15,user_factor_16,user_factor_17,user_factor_18,user_factor_19,user_factor_20
0,1,820165,12.0,1.5,NaN,54.918462,106,225,1428,12.784615,...,-1.431015,-6.023457,-4.243896,11.608219,12.846864,3.189416,4.980285,-7.749927,1.946641,1.528602
1,1,823721,13.0,4.0,NaN,54.918462,96,225,1428,12.784615,...,-1.431015,-6.023457,-4.243896,11.608219,12.846864,3.189416,4.980285,-7.749927,1.946641,1.528602


In [37]:
targets_test = get_targets_lvl_2(data_train, data_valid, user_item_features, N, add_to_lvl_2)

print(f'число пользователей: {targets_test.user_id.nunique()}')
print(f'среднее число покупок: {round(targets_test["target"].mean(), 4)}')

targets_test.head(2)


число пользователей: 2272
среднее число покупок: 0.0781


,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,manufacturer_freq,...,user_factor_11,user_factor_12,user_factor_13,user_factor_14,user_factor_15,user_factor_16,user_factor_17,user_factor_18,user_factor_19,user_factor_20
0,84,1001646,0.0,1225,GROCERY,1,CAT FOOD,CANNED CAT FOOD (9 LIVES/FRISK,5.5 OZ,382.0,...,-4.964427,10.565506,0.068612,-4.39837,-4.874898,9.117214,-10.737057,-0.070772,5.188995,9.173296
1,84,829722,0.0,70,GROCERY,1,LAUNDRY DETERGENTS,LIQUID LAUNDRY DETERGENTS,32 LOAD,162.0,...,-4.964427,10.565506,0.068612,-4.39837,-4.874898,9.117214,-10.737057,-0.070772,5.188995,9.173296


#### Генерация прогнозов

In [38]:
model_lgb = run_model_lgb(targets_test) 

#[2908]	training's auc: 0.964629	valid_1's auc: 0.896782

Training until validation scores don't improve for 30 rounds
[1000]	training's auc: 0.935529	valid_1's auc: 0.888813
[2000]	training's auc: 0.955493	valid_1's auc: 0.894568
Early stopping, best iteration is:
[2908]	training's auc: 0.964629	valid_1's auc: 0.896782


In [39]:
model_cb = run_model_cb(targets_test) 
# bestTest = 0.9076711973
# bestIteration = 2693

0:	test: 0.7811897	best: 0.7811897 (0)	total: 4.41s	remaining: 6h 7m 3s
1000:	test: 0.8947586	best: 0.8947586 (1000)	total: 22m 57s	remaining: 1h 31m 41s
2000:	test: 0.9051848	best: 0.9051848 (2000)	total: 45m 1s	remaining: 1h 7m 29s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.9076711973
bestIteration = 2693

Shrink model to first 2694 iterations.


In [40]:
predictions_lgb_test = model_lgb.predict(targets_test[SELECTED_FEATURES_NAMES].fillna(0))
predictions_cb_test = model_cb.predict_proba(targets_test[SELECTED_FEATURES_NAMES_cb].fillna(0))[:, 1]

Объединение прогнозов

In [42]:
preds_test = pd.DataFrame(zip(predictions_lgb_test, predictions_cb_test),columns=['lgb', 'cb']).mean(axis=1).values
roc_auc_score(targets_test['target'], preds_test)
# 0.955686482496358

0.955686482496358

#### Обработка прогнозов

In [43]:
result_test = get_results(test, targets_test, 
                           predictions_lgb_test,
                           predictions_cb_test, 
                           preds_test)
result_test.head(2)

,user_id,actual,lgb_recommendations,cb_recommendations,cb_lgb_recommendations
0,1,"[880007, 883616, 931136, 938004, 940947, 94726...","[8293439, 1082185, 1074612, 856942, 995242]","[1082185, 8293439, 1074612, 995242, 9297615]","[1082185, 8293439, 1074612, 995242, 9297615]"
1,2,"[820165, 820291, 826784, 826835, 829009, 85784...","[838136, 857849, 1106523, 1133018, 1053690]","[1133018, 1053690, 899624, 1035843, 857849]","[1133018, 857849, 1053690, 838136, 899624]"


####  Метрики качества

#### LightGBM

In [44]:
result_test.apply(lambda row: precision_at_k(row['lgb_recommendations'], row['actual'], 5), axis=1).mean()
# 0.3131034482758621

0.3131034482758621

#### CatBooost

In [46]:
result_test.apply(lambda row: precision_at_k(row['cb_recommendations'], row['actual'], 5), axis=1).mean()
# 0.33389920424403186

0.33389920424403186

#### Ensemble

In [47]:
result_test.apply(lambda row: precision_at_k(row['cb_lgb_recommendations'], row['actual'], 5), axis=1).mean()
# 0.3335809018567639

0.3335809018567639

#### Запись результатов

In [48]:
PATH = 'C:/Users/ASER/Desktop/GeekBrains/RecSys/project/'
df = result_test[['user_id', 'cb_recommendations']].copy()
df.to_csv(PATH + 'predictions/predictions_cb_3338.csv', index=False)
df.head(2)

,user_id,cb_recommendations
0,1,"[1082185, 8293439, 1074612, 995242, 9297615]"
1,2,"[1133018, 1053690, 899624, 1035843, 857849]"


In [49]:
models = [model_cb, model_lgb]
for i, name in enumerate(['model_cb.pkl', 'model_lgb.pkl']):
    filename = PATH + '/models/' + '3338_' + name
    pickle.dump(models[i], open(filename, 'wb'))

In [50]:
targets_test.to_csv(PATH + 'data_for_prediction/3338_targets_test.csv')